<a href="https://colab.research.google.com/github/NguyenVanVuong613/PDF_Etracxt/blob/main/Copy_of_mineru_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# minerU install and use demo

## 1.Install

### 1.1Using CPU

In [ ]:
!pip install -U magic-pdf[full] --extra-index-url https://wheels.myhloli.com

In [ ]:
!magic-pdf -v

In [ ]:
#!wget https://github.com/opendatalab/MinerU/raw/master/docs/download_models_hf.py && pip install huggingface_hub

In [ ]:
#from huggingface_hub import snapshot_download
#model_dir = snapshot_download('opendatalab/PDF-Extract-Kit')

In [ ]:
!git lfs install

Git LFS initialized.


In [ ]:
!git lfs clone https://huggingface.co/opendatalab/PDF-Extract-Kit

In [ ]:
"""
move magic-pdf.template.json under root/
modify "models-dir" in magic-pdf.json
"""
!wget https://github.com/opendatalab/MinerU/raw/master/magic-pdf.template.json && mv magic-pdf.template.json ~/magic-pdf.json && sed -i 's|/tmp/models|{model_dir}|g' ~/magic-pdf.json

### 1.2Using GPU

In [ ]:
!sed -i 's|cpu|cuda|g' ~/magic-pdf.json

In [ ]:
!pip install paddlepaddle-gpu==3.0.0b1 -i https://www.paddlepaddle.org.cn/packages/stable/cu118/

## 2.Usage

### 2.1Command line

#### **Convert Single PDF**

In [ ]:
!wget https://github.com/opendatalab/MinerU/raw/master/demo/demo1.pdf && wget https://github.com/opendatalab/MinerU/raw/master/demo/demo2.pdf

In [ ]:
!magic-pdf -p /content/test3.pdf -o output2/ -m auto

In [ ]:
import os
from PIL import Image
from loguru import logger

from magic_pdf.model import pdf_extract_kit

output_dir = '/content/output_formulas'  # This is within Colab's file system

# Create the directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Save mf_image_list to the output directory
for idx, img in enumerate(mf_image_list):
    image_filename = os.path.join(output_dir, f"formula_{idx}.png")
    img.save(image_filename)
    print(f"Saved formula image to {image_filename}")


In [ ]:
import fitz  # PyMuPDF
import matplotlib.pyplot as plt


import fitz  # PyMuPDF
import matplotlib.pyplot as plt
import numpy as np

# Open the PDF file
# Open the PDF file
layout = "output/demo1/auto/layout.pdf"
spans = "output/demo1/auto/spans.pdf"
# Load and render the first page
layout_page_0 = fitz.open(layout).load_page(0)
spans_page_0 = fitz.open(spans).load_page(0)


pix1 = layout_page_0.get_pixmap()
img_array1 = np.frombuffer(pix1.samples, dtype=np.uint8).reshape(pix1.height, pix1.width, pix1.n)

# Load and render the second page
pix2 = spans_page_0.get_pixmap()
img_array2 = np.frombuffer(pix2.samples, dtype=np.uint8).reshape(pix2.height, pix2.width, pix2.n)

# Create subplots
fig, axes = plt.subplots(1, 2, figsize=(12, 8))

# Display the first image
axes[0].imshow(img_array1)
axes[0].axis('off')

# Display the second image
axes[1].imshow(img_array2)
axes[1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
!pip install pypandoc reportlab pdfplumber


In [ ]:
# Install necessary Python libraries
!pip install pypandoc reportlab pdfplumber

# Install wkhtmltopdf as the PDF engine
!apt-get install -y wkhtmltopdf



In [ ]:
# Import necessary libraries
import pypandoc
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
import pdfplumber

# Convert markdown (demo1.md) to a new PDF using pypandoc
md_input = "/content/output/demo1/auto/demo1.md"
pdf_output = "/content/output/demo1/auto/reconstructed_pdf_from_md.pdf"

# Convert markdown to PDF
pypandoc.convert_file(md_input, 'pdf', outputfile=pdf_output)
print(f"PDF generated from markdown: {pdf_output}")

# Open and extract content from demo1_layout.pdf using pdfplumber
layout_pdf_path = "/content/output/demo1/auto/demo1_layout.pdf"
new_pdf_with_layout = "/content/output/demo1/auto/new_pdf_with_layout.pdf"

with pdfplumber.open(layout_pdf_path) as pdf:
    first_page = pdf.pages[0]  # Example: Get the first page
    extracted_text = first_page.extract_text()  # Extract text from the page
    print("Extracted Text from Layout PDF:")
    print(extracted_text)

    # Generate a new PDF by combining markdown content with layout content
    c = canvas.Canvas(new_pdf_with_layout, pagesize=A4)

    # Add text from markdown (already converted to PDF)
    c.drawString(100, 800, "Text extracted from markdown...")

    # Add content extracted from the layout PDF
    c.drawString(100, 750, "Text extracted from layout PDF:")
    c.drawString(100, 725, extracted_text[:1000])  # Add a portion of the extracted text

    # You can add more content such as images, tables, etc., from layout or other sources here.

    c.showPage()
    c.save()

print(f"New PDF with content from layout and markdown: {new_pdf_with_layout}")


#### **Convert Batch PDF**

In [ ]:
!find ./ -name "*.pdf" | xargs -I {} magic-pdf -p {} -o output/

### 2.2API

#### **Simplest Convertion**

In [ ]:
"""
convert pdf to markdown and text
"""
from magic_pdf.rw.DiskReaderWriter import DiskReaderWriter
from magic_pdf.pipe.UNIPipe import UNIPipe
import os
pdf_path = "demo1.pdf"
output_dir = "output_api/"
pdf_name = os.path.basename(pdf_path).split(".")[0]
output_path = os.path.join(output_dir, pdf_name)
output_image_path = os.path.join(output_path, 'images')


image_writer = DiskReaderWriter(output_image_path)
image_dir = str(os.path.basename(output_image_path))
jso_useful_key = {"_pdf_type": "", "model_list": []}
pdf_bytes = open(pdf_path, "rb").read()
pipe = UNIPipe(pdf_bytes, jso_useful_key, image_writer)
pipe.pipe_classify()
pipe.pipe_analyze()
pipe.pipe_parse()
md_content = pipe.pipe_mk_markdown(image_dir, drop_mode="none")
print(md_content)

#### **Convertion with middle details**

In [ ]:
"""
convert pdf and save middle.json model.json
"""
import copy
import json
def json_md_dump(
        pipe,
        md_writer,
        pdf_name,
        content_list,
        md_content,
):
    # 写入模型结果到 model.json
    orig_model_list = copy.deepcopy(pipe.model_list)
    md_writer.write(
        content=json.dumps(orig_model_list, ensure_ascii=False, indent=4),
        path=f"{pdf_name}_model.json"
    )

    # 写入中间结果到 middle.json
    md_writer.write(
        content=json.dumps(pipe.pdf_mid_data, ensure_ascii=False, indent=4),
        path=f"{pdf_name}_middle.json"
    )

    # text文本结果写入到 conent_list.json
    md_writer.write(
        content=json.dumps(content_list, ensure_ascii=False, indent=4),
        path=f"{pdf_name}_content_list.json"
    )

    # 写入结果到 .md 文件中
    md_writer.write(
        content=md_content,
        path=f"{pdf_name}.md"
    )



md_writer = DiskReaderWriter(output_path)
image_path_parent = os.path.basename(output_image_path)
content_list = pipe.pipe_mk_uni_format(image_path_parent, drop_mode="none")
json_md_dump(pipe, md_writer, pdf_name, content_list, md_content)

#### **Produce Equtation Results and draw bounding boxes with number**

In [ ]:
"""
detect and recognize equations from a pdf page
"""

import fitz  # PyMuPDF
import matplotlib.pyplot as plt


import fitz  # PyMuPDF
import matplotlib.pyplot as plt
import numpy as np
def show_pdf(page0, page1):

  pix1 = page0.get_pixmap()
  img_array1 = np.frombuffer(pix1.samples, dtype=np.uint8).reshape(pix1.height, pix1.width, pix1.n)

  # Load and render the second page
  pix2 = page1.get_pixmap()
  img_array2 = np.frombuffer(pix2.samples, dtype=np.uint8).reshape(pix2.height, pix2.width, pix2.n)

  # Create subplots
  fig, axes = plt.subplots(1, 2, figsize=(12, 8))

  # Display the first image
  axes[0].imshow(img_array1)
  axes[0].axis('off')

  # Display the second image
  axes[1].imshow(img_array2)
  axes[1].axis('off')

  plt.tight_layout()
  plt.show()

In [ ]:
"""
output equations in latex and draw bounding boxes with number on a pdf page
"""
from magic_pdf.libs.Constants import CROSS_PAGE
from magic_pdf.libs.commons import fitz  # PyMuPDF
from magic_pdf.libs.ocr_content_type import ContentType, BlockType, CategoryId
from magic_pdf.model.magic_model import MagicModel
from magic_pdf.libs.draw_bbox import draw_bbox_with_number

pdf_info = pipe.pdf_mid_data["pdf_info"]
text_list = []
inline_equation_list = []
interline_equation_list = []
interline_equation_content_list = []
image_list = []
table_list = []
dropped_list = []
next_page_text_list = []
next_page_inline_equation_list = []

def get_span_info(span):
    if span["type"] == ContentType.Text:
        if span.get(CROSS_PAGE, False):
            next_page_text_list.append(span["bbox"])
        else:
            page_text_list.append(span["bbox"])
    elif span["type"] == ContentType.InlineEquation:
        if span.get(CROSS_PAGE, False):
            next_page_inline_equation_list.append(span["bbox"])
        else:
            page_inline_equation_list.append(span["bbox"])
    elif span["type"] == ContentType.InterlineEquation:
        page_interline_equation_list.append(span["bbox"])
        page_interline_equation_content_list.append(span["content"])
    elif span["type"] == ContentType.Image:
        page_image_list.append(span["bbox"])
    elif span["type"] == ContentType.Table:
        page_table_list.append(span["bbox"])

for page in pdf_info:
    page_text_list = []
    page_inline_equation_list = []
    page_interline_equation_list = []
    page_interline_equation_content_list = []
    page_image_list = []
    page_table_list = []
    page_dropped_list = []

    # 将跨页的span放到移动到下一页的列表中
    if len(next_page_text_list) > 0:
        page_text_list.extend(next_page_text_list)
        next_page_text_list.clear()
    if len(next_page_inline_equation_list) > 0:
        page_inline_equation_list.extend(next_page_inline_equation_list)
        next_page_inline_equation_list.clear()

    # 构造dropped_list
    for block in page["discarded_blocks"]:
        if block["type"] == BlockType.Discarded:
            for line in block["lines"]:
                for span in line["spans"]:
                    page_dropped_list.append(span["bbox"])
    dropped_list.append(page_dropped_list)
    # 构造其余useful_list
    for block in page["para_blocks"]:
        if block["type"] in [
            BlockType.Text,
            BlockType.Title,
            BlockType.InterlineEquation,
        ]:
            for line in block["lines"]:
                for span in line["spans"]:
                    get_span_info(span)
        elif block["type"] in [BlockType.Image, BlockType.Table]:
            for sub_block in block["blocks"]:
                for line in sub_block["lines"]:
                    for span in line["spans"]:
                        get_span_info(span)
    text_list.append(page_text_list)
    inline_equation_list.append(page_inline_equation_list)
    interline_equation_list.append(page_interline_equation_list)
    interline_equation_content_list.append(page_interline_equation_content_list)
    image_list.append(page_image_list)
    table_list.append(page_table_list)

In [ ]:
pdf_docs = fitz.open("pdf", pdf_bytes)
page_no = 3
page = pdf_docs[page_no]
draw_bbox_with_number(page_no, interline_equation_list, page, [0, 0, 255], False)
ori_pdf = "demo1.pdf"
# Load and render the first page
ori_page_i = fitz.open(ori_pdf).load_page(page_no)
show_pdf(ori_page_i, page)
for i, equation in enumerate(interline_equation_content_list[page_no]):
  print(f"{i}: {equation}")

#### **Produce image Results and draw bounding boxes with number**

In [ ]:
pdf_docs = fitz.open("pdf", pdf_bytes)
page_no = 3
page = pdf_docs[page_no]
draw_bbox_with_number(page_no, image_list, page, [0, 0, 255], False)
ori_pdf = "demo1.pdf"
# Load and render the first page
ori_page_i = fitz.open(ori_pdf).load_page(page_no)
show_pdf(ori_page_i, page)

#### **Produce text Results and draw bounding boxes with number**

In [ ]:
pdf_docs = fitz.open("pdf", pdf_bytes)
page_no = 3
page = pdf_docs[page_no]
draw_bbox_with_number(page_no, text_list, page, [0, 0, 255], False)
ori_pdf = "demo1.pdf"
# Load and render the first page
ori_page_i = fitz.open(ori_pdf).load_page(page_no)
show_pdf(ori_page_i, page)

#### **Produce Dropped content and draw bounding boxes with number**

In [ ]:
pdf_docs = fitz.open("pdf", pdf_bytes)
page_no = 3
page = pdf_docs[page_no]
draw_bbox_with_number(page_no, dropped_list, page, [0, 0, 255], False)
ori_pdf = "demo1.pdf"
# Load and render the first page
ori_page_i = fitz.open(ori_pdf).load_page(page_no)
show_pdf(ori_page_i, page)